In [2]:
from xml.etree import ElementTree as ET

# Caminho para o arquivo GPX fornecido
gpx_file_path = '/mnt/data/20240409-101422 - Trajeto de teste 1 09_04.gpx'

# Ler o arquivo GPX
with open(gpx_file_path, 'r') as gpx_file:
    gpx_tree = ET.parse(gpx_file)
    root = gpx_tree.getroot()

# GPX files usam namespaces. Vamos encontrar o namespace correto para acessar os elementos
namespace = {'gpx': 'http://www.topografix.com/GPX/1/0'}

# Vamos tentar encontrar elementos comuns como trk (track) que geralmente contém a informação de trajeto
tracks = root.findall('gpx:trk', namespace)

# Para cada trilha (track), vamos tentar extrair um resumo das informações disponíveis, como nome e número de segmentos
tracks_info = []
for trk in tracks:
    trk_info = {
        'name': trk.find('gpx:name', namespace).text if trk.find('gpx:name', namespace) is not None else 'Unnamed',
        'segments': len(trk.findall('gpx:trkseg', namespace))
    }
    tracks_info.append(trk_info)

tracks_info, len(tracks)

([{'name': 'Trajeto 20240409-101422', 'segments': 1}], 1)

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Encontrar todos os pontos de trilha (trkpt) dentro do único segmento de trilha (trkseg)
trkseg = tracks[0].find('gpx:trkseg', namespace)
trkpts = trkseg.findall('gpx:trkpt', namespace)

# Extrair informações de cada ponto de trilha
trkpt_data = []
for pt in trkpts:
    # Coordenadas
    lat = pt.attrib['lat']
    lon = pt.attrib['lon']
    
    # Elevação
    ele = pt.find('gpx:ele', namespace).text if pt.find('gpx:ele', namespace) is not None else None
    
    # Tempo
    time = pt.find('gpx:time', namespace).text if pt.find('gpx:time', namespace) is not None else None
    
    trkpt_data.append({
        'latitude': lat,
        'longitude': lon,
        'elevation': ele,
        'time': datetime.strptime(time, '%Y-%m-%dT%H:%M:%SZ') if time else None
    })

# Converter para DataFrame
trkpt_df = pd.DataFrame(trkpt_data)

# Vamos ver as primeiras linhas para garantir que tudo foi extraído corretamente
trkpt_df.head(), trkpt_df.shape

(      latitude     longitude elevation                time
 0  -3.74468661  -38.57741007    16.312 2024-04-09 13:14:22
 1  -3.74468491  -38.57741068    16.476 2024-04-09 13:14:23
 2  -3.74468784  -38.57741192    16.121 2024-04-09 13:14:24
 3  -3.74468931  -38.57741166    15.629 2024-04-09 13:14:25
 4  -3.74469007  -38.57741203    15.846 2024-04-09 13:14:26,
 (836, 4))

In [4]:
# Definir o caminho do arquivo de destino
caminho_arquivo_csv = 'gps.csv' # Ajuste o caminho conforme necessário
# Exportar o DataFrame para um arquivo CSV
trkpt_df.to_csv(caminho_arquivo_csv, sep=';', encoding='ISO-8859-1', index=False)

In [5]:
# Separar a coluna 'time' em duas colunas: 'date' e 'time_adjusted'
# Também ajustar a hora diminuindo 3 horas
trkpt_df['date'] = trkpt_df['time'].dt.date
trkpt_df['time_adjusted'] = trkpt_df['time'].dt.time - timedelta(hours=3)

# Remover a coluna 'time' original
trkpt_df.drop('time', axis=1, inplace=True)

# Reordenar colunas para colocar 'date' e 'time_adjusted' primeiro
trkpt_df = trkpt_df[['date', 'time_adjusted', 'latitude', 'longitude', 'elevation']]

# Verificar as alterações com as primeiras linhas do DataFrame
trkpt_df.head(), trkpt_df.shape

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'Timedelta'